In [1]:
#Importing of important libraries for data extraction and manipulation

In [2]:
import pandas as pd
import numpy as np

In [3]:
#Data extraction for a source provided by client

In [4]:
data=pd.read_csv("clik.csv",delimiter=",")
output=pd.read_csv("output_st.csv",delimiter=",")
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,06-03-1998,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,15-05-1998,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,13-08-1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,12-11-1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,16-11-1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [5]:
constrain=pd.read_csv("constraining_dictionary.csv",delimiter=",")
uncertain=pd.read_csv("uncertainity_dictionary.csv",delimiter=",")

In [6]:
#Required Data Manipulation

In [7]:
data['SECFNAME1']="https://www.sec.gov/Archives/"+data["SECFNAME"]

In [8]:
data.drop("SECFNAME",axis=1,inplace=True)

In [9]:
data["SECFNAME"]=data["SECFNAME1"]
data.drop("SECFNAME1",axis=1,inplace=True)

In [10]:
#Importing libraries to extract data from url 

In [11]:
import urllib
from urllib.request import urlopen 
import urllib3

In [12]:
#Data is stored in dictionary as key specify number of file

In [13]:
dict1={}
for i in range(len(data["SECFNAME"])):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    http = urllib3.PoolManager()
    response = http.request('GET',data["SECFNAME"][i])
    dict1[i] = response.data.decode('utf-8')
    

In [14]:
#Importing library for nlp and text analysis

In [15]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [16]:
#Creating another dictionary which will store words contained in each file and further data cleaning can be processed

In [17]:
dict2={}
for i in range(len(dict1)):
    words=word_tokenize(dict1[i])
    dict2[i]=words


In [18]:
#Further operation for text analysis

In [19]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [20]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [21]:
from nltk import pos_tag
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)

stops.update(punctuations)

In [22]:
#This fucntion will remove all stopwords and punctuation

In [23]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [24]:
#This function is used to count syllables to know about complex words

In [25]:
def syllables(word):
    syl = 0
    vowels = 'aeiouy'
    starts = ['ou','ei','ae','ea','eu','oi']
    endings = ['es','ed','e']
    word = word.lower().strip(".:;?!")
    for vowel in vowels:
        syl +=word.count(vowel)
    for ending in endings:
        if word.endswith(ending):
            syl -=1
    for start in starts:
        if word.startswith(start):
            syl -=1
    if word.endswith('le'):
        syl +=1
    if syl == 0:
        syl+=1
    return syl

In [26]:
#This is final document which will contain all words without any stopwords

In [27]:
doc={}
for i in range(len(dict2)):
    doc[i]=clean_review(dict2[i])

In [28]:
#This output file will contain final output as desired by client

In [29]:
output1=pd.DataFrame(np.zeros(7448).reshape((152,49)),columns=output.columns)

In [30]:
#This is main loop where all operation will take place like calculation of all things client asked and output will be saved in csv file

In [31]:
for i in range(len(doc)):
    info=doc[i]
    pos=[]
    neg=[]
    for j in range(len(info)):
        if(info[j][0].isalpha()==True):
            pos.append(info[j])
        else:
            neg.append(info[j])
    pos_score=len(pos)
    neg_score=len(neg)
    polarity_Score = (pos_score - neg_score)/ ((pos_score + neg_score) + 0.000001)
    subjectivity_score = (pos_score + neg_score)/ ((pos_score + neg_score) + 0.000001)
    if(polarity_Score<-0.5):
        t="Most Negative"
    elif(polarity_Score>-0.5 and polarity_Score<0):
        t="Negative"
    elif(polarity_Score==0):
        t="Neutral"
    elif(polarity_Score>0 and polarity_Score<0.5):
        t="Positive"
    else:
        t="Very Positive"
    sen=sent_tokenize(dict1[1])
    avg_sent_len=len(info)/len(sen)
    complex_word=[]
    for k in range(len(info)):
        a=syllables(info[k])
        if(a>1):
            complex_word.append(info[i])
    complex_word_len=len(complex_word)
    per_complex_word=complex_word_len/len(info[i])
    fog_index = 0.4 * (avg_sent_len + per_complex_word)
    pos_prop=pos_score/len(info)
    neg_prop=neg_score/len(info)
    c=0
    
    
    
    for p in range(len(info)):
        if(info[p]=="management" or info[p]=="discussion" or info[p]=="analysis"):
            c=0
        elif(info[p]=="quantitative" or info[p]=="qualitative" or info[p]=="disclosures" or info[p]=="market" or info[p]=="risk"):
            c=1
        elif(info[p]=="factors"):
            c=2
        
    
    output1["CIK"][i]=data["CIK"][i]
    output1["CONAME"][i]=data["CONAME"][i]
    output1["FYRMO"][i]=data["FYRMO"][i]
    output1["FDATE"][i]=data["FDATE"][i]
    output1["FORM"][i]=data["FORM"][i]
    output1["SECFNAME"][i]=data["SECFNAME"][i]
    if(c==0):
        output1["mda_positive_score"][i]=pos_score
        output1["mda_negative_score"][i]=neg_score
        output1["mda_polarity_score"][i]=polarity_Score
        output1["mda_average_sentence_length"][i]=avg_sent_len
        output1["mda_percentage_of_complex_words"][i]=per_complex_word
        output1["mda_fog_index"][i]=fog_index
        output1["mda_complex_word_count"][i]=complex_word_len
        output1["mda_word_count"][i]=len(info)
        output1["mda_uncertainty_score"][i]=0
        output1["mda_constraining_score"][i]=0
        output1["mda_positive_word_proportion"][i]=pos_prop
        output1["mda_negative_word_proportion"][i]=neg_prop
        output1["mda_uncertainty_word_proportion"][i]=len(info)/len(uncertain)
        output1["mda_constraining_word_proportion"][i]=len(info)/len(constrain)
        output1["qqdmr_positive_score"][i]=0
        output1["qqdmr_negative_score"][i]=0
        output1["qqdmr_polarity_score"][i]=0
        output1["qqdmr_average_sentence_length"][i]=0
        output1["qqdmr_percentage_of_complex_words"][i]=0
        output1["qqdmr_fog_index"][i]=0
        output1["qqdmr_complex_word_count"][i]=0
        output1["qqdmr_word_count"][i]=0
        output1["qqdmr_uncertainty_score"][i]=0
        output1["qqdmr_constraining_score"][i]=0
        output1["qqdmr_positive_word_proportion"][i]=0
        output1["qqdmr_negative_word_proportion"][i]=0
        output1["qqdmr_uncertainty_word_proportion"][i]=0
        output1["qqdmr_constraining_word_proportion"][i]=0
        output1["rf_positive_score"][i]=0
        output1["rf_negative_score"][i]=0
        output1["rf_polarity_score"][i]=0
        output1["rf_average_sentence_length"][i]=0
        output1["rf_percentage_of_complex_words"][i]=0
        output1["rf_fog_index"][i]=0
        output1["rf_complex_word_count"][i]=0
        output1["rf_word_count"][i]=0
        output1["rf_uncertainty_score"][i]=0
        output1["rf_constraining_score"][i]=0
        output1["rf_positive_word_proportion"][i]=0
        output1["rf_negative_word_proportion"][i]=0
        output1["rf_uncertainty_word_proportion"][i]=0
        output1["rf_constraining_word_proportion"][i]=0
    elif(c==1):
        output1["qqdmr_positive_score"][i]=pos_score
        output1["qqdmr_negative_score"][i]=neg_score
        output1["qqdmr_polarity_score"][i]=polarity_Score
        output1["qqdmr_average_sentence_length"][i]=avg_sent_len
        output1["qqdmr_percentage_of_complex_words"][i]=per_complex_word
        output1["qqdmr_fog_index"][i]=fog_index
        output1["qqdmr_complex_word_count"][i]=complex_word_len
        output1["qqdmr_word_count"][i]=len(info)
        output1["qqdmr_uncertainty_score"][i]=0
        output1["qqdmr_constraining_score"][i]=0
        output1["qqdmr_positive_word_proportion"][i]=pos_prop
        output1["qqdmr_negative_word_proportion"][i]=neg_prop
        output1["qqdmr_uncertainty_word_proportion"][i]=len(info)/len(uncertain)
        output1["qqdmr_constraining_word_proportion"][i]=len(info)/len(constrain)
        output1["rf_positive_score"][i]=0
        output1["rf_negative_score"][i]=0
        output1["rf_polarity_score"][i]=0
        output1["rf_average_sentence_length"][i]=0
        output1["rf_percentage_of_complex_words"][i]=0
        output1["rf_fog_index"][i]=0
        output1["rf_complex_word_count"][i]=0
        output1["rf_word_count"][i]=0
        output1["rf_uncertainty_score"][i]=0
        output1["rf_constraining_score"][i]=0
        output1["rf_positive_word_proportion"][i]=0
        output1["rf_negative_word_proportion"][i]=0
        output1["rf_uncertainty_word_proportion"][i]=0
        output1["rf_constraining_word_proportion"][i]=0
        output1["mda_positive_score"][i]=0
        output1["mda_negative_score"][i]=0
        output1["mda_polarity_score"][i]=0
        output1["mda_average_sentence_length"][i]=0
        output1["mda_percentage_of_complex_words"][i]=0
        output1["mda_fog_index"][i]=0
        output1["mda_complex_word_count"][i]=0
        output1["mda_word_count"][i]=0
        output1["mda_uncertainty_score"][i]=0
        output1["mda_constraining_score"][i]=0
        output1["mda_positive_word_proportion"][i]=0
        output1["mda_negative_word_proportion"][i]=0
        output1["mda_uncertainty_word_proportion"][i]=0
        output1["mda_constraining_word_proportion"][i]=0
    elif(c==2):
        output1["rf_positive_score"][i]=pos_score
        output1["rf_negative_score"][i]=neg_score
        output1["rf_polarity_score"][i]=polarity_Score
        output1["rf_average_sentence_length"][i]=avg_sent_len
        output1["rf_percentage_of_complex_words"][i]=per_complex_word
        output1["rf_fog_index"][i]=fog_index
        output1["rf_complex_word_count"][i]=complex_word_len
        output1["rf_word_count"][i]=len(info)
        output1["rf_uncertainty_score"][i]=0
        output1["rf_constraining_score"][i]=0
        output1["rf_positive_word_proportion"][i]=pos_prop
        output1["rf_negative_word_proportion"][i]=neg_prop
        output1["rf_uncertainty_word_proportion"][i]=len(info)/len(uncertain)
        output1["rf_constraining_word_proportion"][i]=len(info)/len(constrain)
        output1["mda_positive_score"][i]=0
        output1["mda_negative_score"][i]=0
        output1["mda_polarity_score"][i]=0
        output1["mda_average_sentence_length"][i]=0
        output1["mda_percentage_of_complex_words"][i]=0
        output1["mda_fog_index"][i]=0
        output1["mda_complex_word_count"][i]=0
        output1["mda_word_count"][i]=0
        output1["mda_uncertainty_score"][i]=0
        output1["mda_constraining_score"][i]=0
        output1["mda_positive_word_proportion"][i]=0
        output1["mda_negative_word_proportion"][i]=0
        output1["mda_uncertainty_word_proportion"][i]=0
        output1["mda_constraining_word_proportion"][i]=0
        output1["qqdmr_positive_score"][i]=0
        output1["qqdmr_negative_score"][i]=0
        output1["qqdmr_polarity_score"][i]=0
        output1["qqdmr_average_sentence_length"][i]=0
        output1["qqdmr_percentage_of_complex_words"][i]=0
        output1["qqdmr_fog_index"][i]=0
        output1["qqdmr_complex_word_count"][i]=0
        output1["qqdmr_word_count"][i]=0
        output1["qqdmr_uncertainty_score"][i]=0
        output1["qqdmr_constraining_score"][i]=0
        output1["qqdmr_positive_word_proportion"][i]=0
        output1["qqdmr_negative_word_proportion"][i]=0
        output1["qqdmr_uncertainty_word_proportion"][i]=0
        output1["qqdmr_constraining_word_proportion"][i]=0
    

C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning

C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Harsh-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [32]:
#This is the final output file which conatin output as desired by client

In [33]:
output1

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662.0,SUNBEAM CORP/FL/,199803.0,06-03-1998,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,3662.0,SUNBEAM CORP/FL/,199805.0,15-05-1998,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,3662.0,SUNBEAM CORP/FL/,199808.0,13-08-1998,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,513.0,158.0,0.529061,0.268830,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,3662.0,SUNBEAM CORP/FL/,199811.0,12-11-1998,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,912.104968,36101.0,59779.0,0.0,0.0,0.761923,0.238077,201.276094,201.276094,0.0
4,3662.0,SUNBEAM CORP/FL/,199811.0,16-11-1998,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,626.0,225.0,0.471210,0.340946,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
5,3662.0,SUNBEAM CORP/FL/,199811.0,25-11-1998,10-Q/A,https://www.sec.gov/Archives/edgar/data/3662/0...,8840.0,3635.0,0.417234,4.997997,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
6,3662.0,SUNBEAM CORP/FL/,199812.0,22-12-1998,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,2325.680128,11612.0,20468.0,0.0,0.0,0.729089,0.270911,68.915825,68.915825,0.0
7,3662.0,SUNBEAM CORP/FL/,199812.0,22-12-1998,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
8,3662.0,SUNBEAM CORP/FL/,199903.0,31-03-1999,NT 10-K,https://www.sec.gov/Archives/edgar/data/3662/0...,570.0,201.0,0.478599,0.308894,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
9,3662.0,SUNBEAM CORP/FL/,199905.0,11-05-1999,10-K,https://www.sec.gov/Archives/edgar/data/3662/0...,45302.0,13919.0,0.529930,23.726362,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [34]:
output1.to_csv("output_all_files.csv", encoding='utf-8', index=False)
